In [14]:
import pandas as pd

# Using a raw string literal (recommended on Windows)
df = pd.read_csv("C:\Users\Armstrong\Documents\Cannabis\michigan\general.csv")


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (2435990292.py, line 4)

In [1]:
#!/usr/bin/env python3
import os
import re
import calendar
import pdfplumber
import pandas as pd

# Configuration
dir_pdfs = r"C:\Users\Armstrong\Documents\Cannabis"
dir_out = r"C:\Users\Armstrong\Documents\Cannabis\michigan"
os.makedirs(dir_out, exist_ok=True)

# Helper: derive Period from filename
def parse_period(fn):
    m = re.search(r"([A-Za-z]+)[ _-]+(\d{4})", fn)
    if not m:
        return fn.rstrip('.pdf')
    mon, yr = m.group(1).capitalize(), m.group(2)
    # normalize full month
    for i, name in enumerate(calendar.month_name):
        if name.lower().startswith(mon.lower()):
            return f"{name} {yr}"
    return f"{mon} {yr}"

# Containers
general_rows = []
plants_rows = []
packages_rows = []
transfers_rows = []

# Patterns
general_label = re.compile(r"^General$", re.IGNORECASE)
plants_header = re.compile(r"Plant State\s+Active\s+Destroyed\s+On Hold", re.IGNORECASE)
packages_header = re.compile(r"Package Type\s+Amount", re.IGNORECASE)
transfers_header = re.compile(r"Transfer Type\s+Amount", re.IGNORECASE)

# Iterate PDFs
for fn in sorted(os.listdir(dir_pdfs)):
    if not fn.lower().endswith('.pdf'):
        continue
    path = os.path.join(dir_pdfs, fn)
    period = parse_period(fn)
    print(f"Processing {fn} → {period}")
    # extract all lines
    lines = []
    with pdfplumber.open(path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                lines.extend(text.splitlines())

    # find General
    for i, line in enumerate(lines):
        if general_label.match(line.strip()):
            # next lines contain date range, then metrics
            # assume metrics on lines i+2..i+5
            try:
                ts = lines[i+2]
                tf = lines[i+3]
                mean = lines[i+4]
                med = lines[i+5]
                # extract numbers
                total_sales = re.search(r"\$([\d,]+\.\d{2})", ts).group(1)
                flower_lbs  = re.search(r"\(lbs\)\s*([\d,]+\.\d{2})", tf).group(1)
                mean_price  = re.search(r"mean\).*?\$([\d,]+\.\d{2})", mean).group(1)
                med_price   = re.search(r"median\).*?\$([\d,]+\.\d{2})", med).group(1)
                general_rows.append({
                    'Period': period,
                    'total_sales': total_sales.replace(',',''),
                    'flower_lbs': flower_lbs.replace(',',''),
                    'mean_price': mean_price.replace(',',''),
                    'median_price': med_price.replace(',','')
                })
            except Exception:
                pass
            break

    # find Plants
    for i, line in enumerate(lines):
        if plants_header.search(line):
            # parse next lines until blank or non-numeric
            j = i+1
            # skip 'As of...' line
            if j < len(lines) and 'As of' in lines[j]: j+=1
            # skip header line
            j+=1
            while j < len(lines) and lines[j].strip():
                parts = re.split(r"\s+", lines[j].strip())
                if len(parts) >= 4:
                    stage = parts[0]
                    active, destroyed, onhold = parts[1:4]
                    plants_rows.append({
                        'Period': period,
                        'Stage': stage,
                        'Active': active.replace(',',''),
                        'Destroyed': destroyed.replace(',',''),
                        'On Hold': onhold.replace(',','')
                    })
                j+=1
            break

    # find Packages
    for i, line in enumerate(lines):
        if packages_header.search(line):
            j = i+1
            while j < len(lines) and lines[j].strip():
                parts = re.split(r"\s+", lines[j].strip(), maxsplit=1)
                if len(parts) == 2:
                    pkg, amt = parts
                    packages_rows.append({
                        'Period': period,
                        'Package Type': pkg,
                        'Amount': amt.replace(',','')
                    })
                j+=1
            break

    # find Transfers
    for i, line in enumerate(lines):
        if transfers_header.search(line):
            j = i+1
            while j < len(lines) and lines[j].strip():
                parts = re.split(r"\s+", lines[j].strip(), maxsplit=1)
                if len(parts) == 2:
                    typ, amt = parts
                    transfers_rows.append({
                        'Period': period,
                        'Transfer Type': typ,
                        'Amount': amt.replace(',','')
                    })
                j+=1
            break

# Write CSVs
pd.DataFrame(general_rows).to_csv(os.path.join(dir_out, 'general.csv'), index=False)
pd.DataFrame(plants_rows).to_csv(os.path.join(dir_out, 'plants.csv'), index=False)
pd.DataFrame(packages_rows).to_csv(os.path.join(dir_out, 'packages.csv'), index=False)
pd.DataFrame(transfers_rows).to_csv(os.path.join(dir_out, 'transfers.csv'), index=False)
print("Done: 4 CSVs written to", dir_out)


Processing April 2022 Monthly Report.pdf → April 2022
Processing April 2023 Monthly Report.pdf → April 2023
Processing April 2024 Monthly Report.pdf → April 2024


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

Processing April_2021_Monthly_Report.pdf → April 2021


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

Processing August 2022 Monthly Report.pdf → August 2022
Processing August 2023 Monthly Report.pdf → August 2023
Processing August 2024 Monthly Report.pdf → August 2024


...

Processing October_2020_Monthly_Report_711022_7.pdf → October 2020


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

Processing October_2021_Monthly_Report_741224_7.pdf → October 2021


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

Processing September 2022 Monthly Report.pdf → September 2022
Processing September 2023 Monthly Report.pdf → September 2023
Processing September 2024 Monthly Report.pdf → September 2024
Processing September_2021_Monthly_Report_737999_7.pdf → September 2021
Done: 4 CSVs written to C:\Users\Armstrong\Documents\Cannabis\michigan
